In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from keras import models

Using TensorFlow backend.


In [2]:
model = models.Sequential()
model.add(Conv2D(16,(3,3),activation='relu',padding='same',input_shape=(128,128,3)))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='softmax'))
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


callback = [EarlyStopping(patience=2,monitor='val_loss'),ModelCheckpoint(filepath='./model.h5',monitor='val_loss',save_best_only=True)]

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)       

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2
)

In [4]:
train_generator=data_generator.flow_from_directory(
    './data/train',
    target_size=(128,128),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)
validation_generator=data_generator.flow_from_directory(
    './data/test',
    target_size=(128,128),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)
test_generator=data_generator.flow_from_directory(
    './data/test',
    target_size=(128,128),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

model.fit(train_generator,epochs=20,steps_per_epoch=600,validation_data=validation_generator,validation_steps=40,callbacks=callback)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Epoch 1/20
600/600 [==============================] - 212s 353ms/step - loss: 9.0732 - accuracy: 0.3468 - val_loss: 1.0963 - val_accuracy: 0.3344
Epoch 2/20
600/600 [==============================] - 209s 349ms/step - loss: 1.0408 - accuracy: 0.4330 - val_loss: 1.0818 - val_accuracy: 0.4220
Epoch 3/20
600/600 [==============================] - 216s 360ms/step - loss: 0.8643 - accuracy: 0.5793 - val_loss: 0.7930 - val_accuracy: 0.6015
Epoch 4/20
600/600 [==============================] - 217s 362ms/step - loss: 0.5963 - accuracy: 0.7512 - val_loss: 0.6982 - val_accuracy: 0.7130
Epoch 5/20
600/600 [==============================] - 217s 362ms/step - loss: 0.3667 - accuracy: 0.8627 - val_loss: 0.3647 - val_accuracy: 0.8457
Epoch 6/20
600/600 [==============================] - 218s 364ms/step - loss: 0.2368 - accuracy: 0.9169 - val_loss: 0.2069 - val_accuracy: 0.9432


In [32]:
model.predict('./data/validation/paper1.png')

AttributeError: 'str' object has no attribute 'ndim'

In [6]:
model_score= model.evaluate_generator(test_generator,steps=20)
print("Model loss",model_score[0])
print("model accuracy",model_score[1])

Model loss 0.13633394241333008
model accuracy 0.9538216590881348


In [21]:
import cv2
import numpy as np
img = cv2.imread('./data/validation/rock6.png')
img = cv2.resize(img,(128,128))
img  = np.reshape(img,[128,128,3])
img = np.expand_dims(img,0)
img.shape
classes = model.predict_classes(img)
print(classes)

[1]


In [22]:
model.save('best_model.h5')